In [56]:
from functools import partial
import os.path

from google.auth.transport.requests import Request
from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError
import numpy as np
from scipy.optimize import curve_fit

SCOPES = ['https://www.googleapis.com/auth/spreadsheets.readonly']
SPREADSHEET_ID = '1JNA2ZuNQuX2TNNFF_vuLC4pIwzz_upBPfuAk1qrCqx4'

In [57]:
creds = None

if os.path.exists('token.json'):
    creds = Credentials.from_authorized_user_file('token.json', SCOPES)
    
if not creds or not creds.valid:
    if creds and creds.expired and creds.refresh_token:
        creds.refresh(Request())
    else:
        flow = InstalledAppFlow.from_client_secrets_file('credentials.json', SCOPES)
        creds = flow.run_local_server(port=0)

    with open('token.json', 'w') as token:
        token.write(creds.to_json())

service = build('sheets', 'v4', credentials=creds)

sheet = service.spreadsheets()

def load_data():
    result = sheet.values().get(spreadsheetId=SPREADSHEET_ID, range='Data!M2:P').execute()
    values = np.array(result.get('values', []), dtype=float)
    return values[:, 0], values[:, 1], values[:, 2]

def store_data(sol, range, winf):
    values = [winf] + list(sol)
    for val in values:
        print(val)
    # sheet.values().update(spreadsheetId=SPREADSHEET_ID, range=range,
    #                       body={'values': values}).execute()

In [58]:
x, weight, fat = load_data()
print(weight)

[326.2 326.  321.4 322.5 319.2 319.6 317.4 317.7 316.5 315.6 316. ]


In [59]:
def f(t, w0, Δ0, c4, Δp1, τ1, Δp2, τ2, winf):
    return winf + (w0 - winf - Δp1/2*np.cos(2*np.pi/7*τ1) - Δp2/2*np.cos(2*np.pi*τ2) - c4)*2**(-5/7*t) + \
        c4*np.exp(Δ0/c4*t) + Δp1/2*np.cos(2*np.pi/7*(t - τ1)) + Δp2/2*np.cos(2*np.pi*(t - τ2))

def e(x, y, sol, winf=None):
    if winf is not None:
        return 1 - np.sum((f(x, *sol, winf) - y)**2) / np.sum(np.array(y)**2)
    else:
        return 1 - np.sum((f(x, *sol) - y)**2) / np.sum(np.array(y)**2)

bounds = ((-np.inf, -np.inf, -np.inf, -np.inf, 0, -np.inf, 0), (np.inf, np.inf, np.inf, np.inf, 7, np.inf, 1))

In [60]:
winf = (1800 - 88.362 + 5.677*43 - 4.799*76*2.54)/13.397*2.2
sol, _ = curve_fit(partial(f, winf=winf), x, weight, bounds=bounds)
err = e(x, weight, sol, winf=winf)
store_data(sol, 'Data!W2', winf=winf)

169.0356115548257
326.3724266365557
-0.5888244047521665
149.78155853247247
2.0456397069547094
6.999999999999996
-1.4327667645004918
0.9999999999999999


In [61]:
winf = 0
sol, _ = curve_fit(partial(f, winf=winf), x, fat, bounds=bounds)
err = e(x, fat, sol, winf=winf)
store_data(sol, 'Data!X2', winf=winf)

0
44.171059031116265
-0.28898591809846996
44.67231282748032
0.36780566738252
2.4676586748111844
0.30330462402442787
0.7126562484038413
